In [1]:
import pandas as pd
from functools import reduce

/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df1 = pd.read_sas('testdata/enrollees.sas7bdat')
df2 = pd.read_sas('testdata/measinventory.sas7bdat')
df3 = pd.read_sas('testdata/outcomes99.sas7bdat')
df4 = pd.read_sas('testdata/subjectchar00.sas7bdat')
df5 = pd.read_sas('testdata/allclinical00.sas7bdat')
df6 = pd.read_sas('testdata/allclinical01.sas7bdat')
print("OAI df1 shape:", df1.shape)
print("OAI df2 shape:", df2.shape)
print("OAI df3 shape:", df3.shape)
print("OAI df4 shape:", df4.shape)
print("OAI df5 shape:", df5.shape)
print("OAI df6 shape:", df6.shape)

OAI df1 shape: (4796, 60)
OAI df2 shape: (4796, 381)
OAI df3 shape: (4796, 92)
OAI df4 shape: (4796, 125)
OAI df5 shape: (4796, 1187)
OAI df6 shape: (4796, 575)


In [3]:
# uppercase all of variables
dfs = [df1, df2, df3, df4, df5, df6]

for df in dfs:
    df.columns = df.columns.str.upper()

##### Variables finding

In [4]:
variables = [
    "V00AGE", 
    "P02SEX", 
    "P01BMI", 
    "V00CESD6", 
    "V00P7LKRCV", 
    "V00P7RKRCV", 
    "V00KOOSQOL", 
    "V00CEMPLOY", 
    "V00PASE", 
    "V00COHORT",
    "V01P7LKRCV",
    "V01P7RKRCV",
    "V01KOOSQOL",
    "V01PASE",
    
    'V00NSAIDS',     #baseline   medication nonprescription
    'V00NSAIDRX',    #baseline   medication prescription
    
]

result = {}

for i, df in enumerate(dfs, start=1):
    df_name = f"df{i}"
    present_vars = [var for var in variables if var in df.columns]
    result[df_name] = present_vars


result

{'df1': ['P02SEX', 'V00COHORT'],
 'df2': ['V00AGE', 'P02SEX'],
 'df3': [],
 'df4': ['V00AGE', 'V00CEMPLOY', 'V00PASE'],
 'df5': ['V00AGE',
  'P01BMI',
  'V00CESD6',
  'V00P7LKRCV',
  'V00P7RKRCV',
  'V00KOOSQOL',
  'V00CEMPLOY',
  'V00PASE',
  'V00NSAIDS',
  'V00NSAIDRX'],
 'df6': ['V01P7LKRCV', 'V01P7RKRCV', 'V01KOOSQOL', 'V01PASE']}

In [5]:
variable_presence = {var: False for var in variables}

for var in variables:
    for df in dfs:
        if var in df.columns:
            variable_presence[var] = True
            break  


variable_presence

{'V00AGE': True,
 'P02SEX': True,
 'P01BMI': True,
 'V00CESD6': True,
 'V00P7LKRCV': True,
 'V00P7RKRCV': True,
 'V00KOOSQOL': True,
 'V00CEMPLOY': True,
 'V00PASE': True,
 'V00COHORT': True,
 'V01P7LKRCV': True,
 'V01P7RKRCV': True,
 'V01KOOSQOL': True,
 'V01PASE': True,
 'V00NSAIDS': True,
 'V00NSAIDRX': True}

##### We understand we need df1 and df5 and df6 (followup variables) which is enrollees and allclinical00

In [6]:
df1 = df1.drop(columns=['VERSION'])
df5 = df5.drop(columns=['VERSION'])

print("OAI df1 shape:", df1.shape)
print("OAI df5 shape:", df5.shape)


OAI df1 shape: (4796, 59)
OAI df5 shape: (4796, 1186)


In [7]:
dfx = df1[["ID", "P02SEX","V00COHORT"]]
dfx

,ID,P02SEX,V00COHORT
0,b'9000099',1.0,1.0
1,b'9000296',1.0,2.0
2,b'9000622',2.0,1.0
3,b'9000798',1.0,1.0
4,b'9001104',2.0,2.0
...,...,...,...
4791,b'9999365',1.0,2.0
4792,b'9999510',1.0,2.0
4793,b'9999862',2.0,2.0
4794,b'9999865',2.0,2.0


In [8]:
dfy = df5[["ID", "V00AGE", "P01BMI", "V00CESD6", "V00P7LKRCV", "V00P7RKRCV", "V00KOOSQOL", "V00CEMPLOY", "V00PASE", 
           
           'V00NSAIDS','V00NSAIDRX',
        ]]
 
dfy

,ID,V00AGE,P01BMI,V00CESD6,V00P7LKRCV,V00P7RKRCV,V00KOOSQOL,V00CEMPLOY,V00PASE,V00NSAIDS,V00NSAIDRX
0,b'9000099',59.0,23.8,1.0,0.0,4.0,25.00,1.0,132.0,1.0,0.0
1,b'9000296',69.0,29.8,1.0,0.0,0.0,100.00,1.0,190.0,0.0,0.0
2,b'9000622',71.0,22.7,1.0,0.0,3.0,50.00,4.0,157.0,0.0,0.0
3,b'9000798',56.0,32.4,2.0,4.0,0.0,43.75,1.0,250.0,1.0,0.0
4,b'9001104',72.0,30.7,1.0,0.0,5.0,37.50,4.0,164.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4791,b'9999365',56.0,30.3,2.0,4.0,6.0,18.75,1.0,200.0,0.0,0.0
4792,b'9999510',50.0,28.9,1.0,3.0,0.0,68.75,1.0,315.0,0.0,0.0
4793,b'9999862',61.0,23.7,1.0,0.0,0.0,93.75,4.0,186.0,0.0,0.0
4794,b'9999865',61.0,25.0,2.0,0.0,0.0,100.00,4.0,95.0,0.0,0.0


In [9]:
dfz = df6[['ID', 'V01P7LKRCV', 'V01P7RKRCV', 'V01KOOSQOL', 'V01PASE' ]]
dfz

,ID,V01P7LKRCV,V01P7RKRCV,V01KOOSQOL,V01PASE
0,b'9000099',0.0,0.0,31.25,119.0
1,b'9000296',0.0,0.0,100.00,190.0
2,b'9000622',0.0,6.0,81.25,118.0
3,b'9000798',5.0,0.0,31.25,285.0
4,b'9001104',0.0,4.0,62.50,157.0
...,...,...,...,...,...
4791,b'9999365',5.0,5.0,56.25,242.0
4792,b'9999510',3.0,0.0,68.75,NaN
4793,b'9999862',0.0,0.0,100.00,215.0
4794,b'9999865',0.0,0.0,100.00,82.0


In [10]:
#Common columns between the dataframes
common_columns = set(dfx.columns) & set(dfy.columns) 
common_columns_list = list(common_columns)
common_columns

{'ID'}

In [11]:
# Merge 3 datasets:
dfs = [dfx, dfy, dfz]
df = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)
print("OAI shape after merging 2 datasets:", df.shape)
df

OAI shape after merging 2 datasets: (4796, 17)


,ID,P02SEX,V00COHORT,V00AGE,P01BMI,V00CESD6,V00P7LKRCV,V00P7RKRCV,V00KOOSQOL,V00CEMPLOY,V00PASE,V00NSAIDS,V00NSAIDRX,V01P7LKRCV,V01P7RKRCV,V01KOOSQOL,V01PASE
0,b'9000099',1.0,1.0,59.0,23.8,1.0,0.0,4.0,25.00,1.0,132.0,1.0,0.0,0.0,0.0,31.25,119.0
1,b'9000296',1.0,2.0,69.0,29.8,1.0,0.0,0.0,100.00,1.0,190.0,0.0,0.0,0.0,0.0,100.00,190.0
2,b'9000622',2.0,1.0,71.0,22.7,1.0,0.0,3.0,50.00,4.0,157.0,0.0,0.0,0.0,6.0,81.25,118.0
3,b'9000798',1.0,1.0,56.0,32.4,2.0,4.0,0.0,43.75,1.0,250.0,1.0,0.0,5.0,0.0,31.25,285.0
4,b'9001104',2.0,2.0,72.0,30.7,1.0,0.0,5.0,37.50,4.0,164.0,0.0,0.0,0.0,4.0,62.50,157.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,b'9999365',1.0,2.0,56.0,30.3,2.0,4.0,6.0,18.75,1.0,200.0,0.0,0.0,5.0,5.0,56.25,242.0
4792,b'9999510',1.0,2.0,50.0,28.9,1.0,3.0,0.0,68.75,1.0,315.0,0.0,0.0,3.0,0.0,68.75,NaN
4793,b'9999862',2.0,2.0,61.0,23.7,1.0,0.0,0.0,93.75,4.0,186.0,0.0,0.0,0.0,0.0,100.00,215.0
4794,b'9999865',2.0,2.0,61.0,25.0,2.0,0.0,0.0,100.00,4.0,95.0,0.0,0.0,0.0,0.0,100.00,82.0


In [12]:
df

,ID,P02SEX,V00COHORT,V00AGE,P01BMI,V00CESD6,V00P7LKRCV,V00P7RKRCV,V00KOOSQOL,V00CEMPLOY,V00PASE,V00NSAIDS,V00NSAIDRX,V01P7LKRCV,V01P7RKRCV,V01KOOSQOL,V01PASE
0,b'9000099',1.0,1.0,59.0,23.8,1.0,0.0,4.0,25.00,1.0,132.0,1.0,0.0,0.0,0.0,31.25,119.0
1,b'9000296',1.0,2.0,69.0,29.8,1.0,0.0,0.0,100.00,1.0,190.0,0.0,0.0,0.0,0.0,100.00,190.0
2,b'9000622',2.0,1.0,71.0,22.7,1.0,0.0,3.0,50.00,4.0,157.0,0.0,0.0,0.0,6.0,81.25,118.0
3,b'9000798',1.0,1.0,56.0,32.4,2.0,4.0,0.0,43.75,1.0,250.0,1.0,0.0,5.0,0.0,31.25,285.0
4,b'9001104',2.0,2.0,72.0,30.7,1.0,0.0,5.0,37.50,4.0,164.0,0.0,0.0,0.0,4.0,62.50,157.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,b'9999365',1.0,2.0,56.0,30.3,2.0,4.0,6.0,18.75,1.0,200.0,0.0,0.0,5.0,5.0,56.25,242.0
4792,b'9999510',1.0,2.0,50.0,28.9,1.0,3.0,0.0,68.75,1.0,315.0,0.0,0.0,3.0,0.0,68.75,NaN
4793,b'9999862',2.0,2.0,61.0,23.7,1.0,0.0,0.0,93.75,4.0,186.0,0.0,0.0,0.0,0.0,100.00,215.0
4794,b'9999865',2.0,2.0,61.0,25.0,2.0,0.0,0.0,100.00,4.0,95.0,0.0,0.0,0.0,0.0,100.00,82.0


In [13]:
df.to_csv('testdata/0.OAI.csv')
